<a href="https://colab.research.google.com/github/078373uhi/078373uhi-DAOTW_A2/blob/main/078373_DAOTW_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Analytics on the Web - Assignment 2
### Michelle Paterson 078373

# Introduction

This report aims to create, test and evaluate a number of Linear Regression and Deep Neural Network models to investigate the relationship between weather variables and traffic collisions in a bid to accurately predict accidents.  This is to allow the better allocation of emergency service staff and resources.  To do so it will use the datasets previously created to train models before testing them to check the accuracy of the predictions made.

# Methodology

First the previously created datasets will be read in.  Those to be used in models will then be split into the main dataset with a portion withheld to be used as a validation dataset to test the completed models.

Linear Regression models will test the linear relationship between visibility and number of accidents, temperature and number of accidents, fog and number of accidents, fog/temperature and number of accidents, and fog/temperature/visibility and number of accidents.

Deep Neural Network models will investigate the complex relationships between visibility and number of accidents, temperature and number of accidents, fog and number of accidents, temperature/visibility and number of accidents, and fog/visibility and number of accidents.



# Results

## Load packages required

In [115]:
import pandas as pd
import numpy as np
import shutil
import sklearn
from sklearn.model_selection import train_test_split
import random

%tensorflow_version 1.x
import tensorflow as tf

print(tf.__version__)


1.15.2


## Linear Regression Models


### Create dataframes from data created previously for linear regressions


In [203]:
df_dow_numacc = pd.read_csv('https://raw.githubusercontent.com/078373uhi/078373uhi-DAOTW_A2/main/lrdata_dow_numacc.csv', index_col=0, )
df_dow_fog_numacc = pd.read_csv('https://raw.githubusercontent.com/078373uhi/078373uhi-DAOTW_A2/main/lrdata_dow_fog_numacc.csv', index_col=0, )
df_dow_temp_numacc = pd.read_csv('https://raw.githubusercontent.com/078373uhi/078373uhi-DAOTW_A2/main/lrdata_dow_temp_numacc.csv', index_col=0, )
df_dow_visib_numacc = pd.read_csv('https://raw.githubusercontent.com/078373uhi/078373uhi-DAOTW_A2/main/lrdata_dow_visib_numacc.csv', index_col=0, )
df_fogvisib_numacc = pd.read_csv('https://raw.githubusercontent.com/078373uhi/078373uhi-DAOTW_A2/main/lrdata_fogvisib_numacc.csv', index_col=0, )
df_tempfog_numacc = pd.read_csv('https://raw.githubusercontent.com/078373uhi/078373uhi-DAOTW_A2/main/lrdata_tempfog_numacc.csv', index_col=0, )
df_tempvisib_numacc = pd.read_csv('https://raw.githubusercontent.com/078373uhi/078373uhi-DAOTW_A2/main/lrdata_tempvisib_numacc.csv', index_col=0, )
df_tempfogvisib_numacc = pd.read_csv('https://raw.githubusercontent.com/078373uhi/078373uhi-DAOTW_A2/main/lrdata_tempfogvisib_numacc.csv', index_col=0, )

**Check dataframes**

In [205]:
print(df_dow_numacc[:10])
print(df_dow_fog_numacc[:10])
print(df_dow_temp_numacc[:10])
print(df_dow_visib_numacc[:10])
print(df_fogvisib_numacc[:10])
print(df_tempfog_numacc[:10])
print(df_tempvisib_numacc[:10])
print(df_tempfogvisib_numacc[:10])

    dayofweek  num_accidents
1           4       0.059633
2           5       0.286697
3           6       0.444954
4           7       0.344037
5           1       0.077982
6           2       0.087156
7           3       0.380734
8           4       0.362385
9           5       0.302752
10          6       0.458716
    dayofweek  fog  num_accidents
1           4    0       0.059633
2           5    0       0.286697
3           6    0       0.444954
4           7    0       0.344037
5           1    0       0.077982
6           2    0       0.087156
7           3    0       0.380734
8           4    0       0.362385
9           5    1       0.302752
10          6    0       0.458716
    dayofweek  temp  num_accidents
1           4  38.0       0.059633
2           5  27.5       0.286697
3           6  21.8       0.444954
4           7  32.2       0.344037
5           1  37.3       0.077982
6           2  35.7       0.087156
7           3  35.9       0.380734
8           4  39.8       0

### Create and test model for visibility

In [118]:
#shuffle visibility dataset 
shuffle_visib = df_dow_visib_numacc.iloc[np.random.permutation(len(df_dow_visib_numacc))]

#create training and validation data
shuffle_visib_train, shuffle_visib_test = train_test_split(shuffle_visib, test_size=0.1)

#select variables used for prediction
predictors_visib = shuffle_visib_train.iloc[:,0:2]

print(predictors_visib[:10])

      dayofweek  visib
950           1   10.0
210           3    6.5
2299          6    9.6
938           3    9.9
789           1   10.0
1152          7    9.5
1612          5    3.6
1693          2    6.3
2394          3    9.5
1277          6    5.9


In [119]:
shuffle_visib_train[:5]

,dayofweek,visib,num_accidents
950,1,10.0,0.397463
210,3,6.5,0.555046
2299,6,9.6,0.412946
938,3,9.9,0.568710
789,1,10.0,0.382664


In [120]:
#create targets 
targets_visib = shuffle_visib_train.iloc[:,2]

print(targets_visib[:10])

950     0.397463
210     0.555046
2299    0.412946
938     0.568710
789     0.382664
1152    0.238197
1612    0.628692
1693    0.352321
2394    0.636161
1277    0.849785
Name: num_accidents, dtype: float64


In [121]:
SCALE_NUM_ACCIDENTS = 1.0

In [122]:
# Training set (80% of the length of the shuffle array)
trainsize = int(len(shuffle_visib_train['num_accidents'])*0.8)
# Test set = 20% of the length of the shuffle array.
testsize = len(shuffle_visib_train['num_accidents']) - trainsize

# Define the number of input values 
nppredictors = 2
# Define the number of output values
noutputs = 1

Create model

In [123]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

shutil.rmtree('/tmp/linear_regression_trained_model_v', ignore_errors=True)

estimator_visib = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_v', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors_visib.values)))

# Starting to train
print("starting to train");

# Train the model. 
estimator_visib.fit(predictors_visib[:trainsize].values, targets_visib[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_ACCIDENTS, steps=10000)

# Check  predictions 
preds = estimator_visib.predict(x=predictors_visib[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_ACCIDENTS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE
rmse = np.sqrt(np.mean((targets_visib[trainsize:].values - predslistscale)**2))
print('LinearRegression Visibility has RMSE of {0}'.format(rmse));

# Calculate the mean of the Number of Accidents.
avg = np.mean(shuffle_visib_train['num_accidents'][:trainsize])

# Calculate the RMSE 
rmse = np.sqrt(np.mean((shuffle_visib_train['num_accidents'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4f9b333dd0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_v', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running

It can be seen that the RMSE values are fairly close here (0.179 and 0.200) which is good so it will be tested with the validation dataset.

Test model with validation data

In [124]:
shuffle_visib_test[:3]

,dayofweek,visib,num_accidents
228,7,10.0,0.740826
469,3,7.0,0.356481
1756,2,10.0,0.331224


In [125]:
input = pd.DataFrame.from_dict(data = 
				{'dayofweek' : [7,3,2],
         'visib' : [10.0, 7.8, 9.4]
        })
					

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_v', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
predslistscale = preds['scores']*850
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4fa079b090>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_v', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_v/mode

Using the first three rows of the shuffled test dataset it can be seen that the expected accident values are 0.743, 0.565 and 0.462.  The actual values returned from the model are 0.685, 0.485 and 0.425.  The first two rows gave a quite good result but the third one was the most accurate.

Test model with random validation data

In [126]:
np.random.seed([2])

shuffle_visib_test.sample(n = 3)

,dayofweek,visib,num_accidents
253,4,10.0,0.628440
990,6,7.8,0.818182
2165,5,10.0,0.567265


In [127]:
input = pd.DataFrame.from_dict(data = 
				{'dayofweek' : [3,1,2],
         'visib' : [3.6, 10, 9.9]
        })
					

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_v', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
predslistscale = preds['scores']*850
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4f9a9ca7d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_v', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_v/mode

Testing the model with a random selection of three rows from the test dataset gives a normalised number of accidents of 0.866, 0.355 and 0.419.  The model returned expected values of 0.503, 0.370 and 0.423.  The first row did not give an accurate result but the second and third rows are much closer.

### Create and test model for temperature

In [128]:
#shuffle temperature dataset 
shuffle_temp = df_dow_temp_numacc.iloc[np.random.permutation(len(df_dow_temp_numacc))]

#create training and validation data
shuffle_temp_train, shuffle_temp_test = train_test_split(shuffle_temp, test_size=0.1)

#select variables used for prediction
predictors_temp = shuffle_temp_train.iloc[:,0:2]

print(predictors_temp[:10])

      dayofweek  temp
1143          5  41.5
1960          3  50.5
678           2  53.2
2180          6  38.6
1288          3  63.1
86            5  41.7
161           3  62.2
1428          3  39.5
1663          7  75.6
928           7  64.9


In [129]:
shuffle_temp_train[:5]

,dayofweek,temp,num_accidents
1143,5,41.5,0.343348
1960,3,50.5,0.650224
678,2,53.2,0.136574
2180,6,38.6,0.912556
1288,3,63.1,0.542918


In [130]:
#create targets 
targets_temp = shuffle_temp_train.iloc[:,2]

print(targets_temp[:10])

1143    0.343348
1960    0.650224
678     0.136574
2180    0.912556
1288    0.542918
86      0.197248
161     0.616972
1428    0.605150
1663    0.763713
928     0.638478
Name: num_accidents, dtype: float64


In [ ]:
SCALE_NUM_ACCIDENTS = 1.0

In [131]:
# Training set (80% of the length of the shuffle array)
trainsize = int(len(shuffle_temp_train['num_accidents'])*0.8)
# Test set = 20% of the length of the shuffle array.
testsize = len(shuffle_temp_train['num_accidents']) - trainsize

# Define the number of input values 
nppredictors = 2
# Define the number of output values
noutputs = 1

Create model

In [132]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

shutil.rmtree('/tmp/linear_regression_trained_model_t', ignore_errors=True)

estimator_temp = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_t', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors_temp.values)))

# Starting to train
print("starting to train");

# Train the model. 
estimator_temp.fit(predictors_temp[:trainsize].values, targets_temp[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_ACCIDENTS, steps=10000)

# Check  predictions 
preds = estimator_temp.predict(x=predictors_temp[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_ACCIDENTS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE
rmse = np.sqrt(np.mean((targets_temp[trainsize:].values - predslistscale)**2))
print('LinearRegression Temperature has RMSE of {0}'.format(rmse));

# Calculate the mean of the Number of Accidents.
avg = np.mean(shuffle_temp_train['num_accidents'][:trainsize])

# Calculate the RMSE 
rmse = np.sqrt(np.mean((shuffle_temp_train['num_accidents'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4fa0b38bd0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_t', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running

The RMSE results are 0.181 and 0.196 which is a good result so the model will now be tested with the validation data.

Test model with validation data

In [133]:
shuffle_temp_test[:3]

,dayofweek,temp,num_accidents
1613,6,55.7,0.597046
570,6,69.6,0.414352
2426,7,72.0,0.477679


In [134]:
input = pd.DataFrame.from_dict(data = 
				{'dayofweek' : [4,7,3],
         'temp' : [75.3, 31.2, 32.1]
        })
					

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_t', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
predslistscale = preds['scores']*850
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4f9a916d50>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_t', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_t/mode

It can be seen that the normalised number of accidents from the test data is 0.693, 0.188 and 0.614 while the results of the model test are actually 0.506, 0.525 and 0.351.  This is much less accurate than the visibility model.

Test model with random data from the validation data

In [135]:
np.random.seed([3])

shuffle_temp_test.sample(n = 3)

,dayofweek,temp,num_accidents
591,6,67.8,0.613426
148,4,56.6,0.759174
12,1,42.7,0.275229


In [136]:
input = pd.DataFrame.from_dict(data = 
				{'dayofweek' : [5,5,3],
         'temp' : [18.2, 59.2, 21.9]
        })
					

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_t', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
predslistscale = preds['scores']*850
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4f9b70a450>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_t', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_t/mode

It can be seen that the normalised number of accidents from the test data are 0.593, 0.693 and 0.888 while the results of the model test are actually 0.404, 0.509 and 0.325 which again is not a particularly accurate result.

### Create and test model for fog

In [137]:
#shuffle fog dataset 
shuffle_fog = df_dow_fog_numacc.iloc[np.random.permutation(len(df_dow_fog_numacc))]

#create training and validation data
shuffle_fog_train, shuffle_fog_test = train_test_split(shuffle_fog, test_size=0.1)

#select variables used for prediction
predictors_fog = shuffle_fog_train.iloc[:,0:2]

print(predictors_fog[:10])

      dayofweek  fog
1588          2    1
1838          7    1
317           5    0
1476          2    0
2000          1    1
481           1    0
2329          1    0
1996          4    1
1842          4    0
2196          1    0


In [138]:
shuffle_fog_train[:5]

,dayofweek,fog,num_accidents
1588,2,1,0.301688
1838,7,1,0.683857
317,5,0,0.545872
1476,2,0,0.251055
2000,1,1,0.255605


In [139]:
#create targets 
targets_fog = shuffle_fog_train.iloc[:,2]

print(targets_fog[:10])

1588    0.301688
1838    0.683857
317     0.545872
1476    0.251055
2000    0.255605
481     0.435185
2329    0.656250
1996    0.926009
1842    0.488789
2196    0.223214
Name: num_accidents, dtype: float64


In [140]:
# Training set (80% of the length of the shuffle array)
trainsize = int(len(shuffle_fog_train['num_accidents'])*0.8)
# Test set = 20% of the length of the shuffle array.
testsize = len(shuffle_fog_train['num_accidents']) - trainsize

# Define the number of input values 
nppredictors = 2
# Define the number of output values
noutputs = 1

Create model

In [141]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

shutil.rmtree('/tmp/linear_regression_trained_model_f', ignore_errors=True)

estimator_fog = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_f', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors_fog.values)))

# Starting to train
print("starting to train");

# Train the model. 
estimator_fog.fit(predictors_fog[:trainsize].values, targets_fog[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_ACCIDENTS, steps=10000)

# Check  predictions 
preds = estimator_fog.predict(x=predictors_fog[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_ACCIDENTS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE
rmse = np.sqrt(np.mean((targets_fog[trainsize:].values - predslistscale)**2))
print('LinearRegression Fog has RMSE of {0}'.format(rmse));

# Calculate the mean of the Number of Accidents.
avg = np.mean(shuffle_fog_train['num_accidents'][:trainsize])

# Calculate the RMSE 
rmse = np.sqrt(np.mean((shuffle_fog_train['num_accidents'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4f9b8fa410>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_f', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running

The RMSE values are close (0.175 and 0.197) so the model will now be tested with the validation data.

Test model using validation data - first three rows.

In [142]:
shuffle_fog_test[:3]

,dayofweek,fog,num_accidents
1341,7,0,0.712446
282,5,0,0.399083
1404,7,0,0.884120


In [143]:
input = pd.DataFrame.from_dict(data = 
				{'dayofweek' : [6,1,2],
         'fog' : [0, 1, 0]
        })
					

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_f', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
predslistscale = preds['scores']*850
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4f9b2c3590>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_f', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_f/mode

The values for number of accidents from the test data are 0.600, 0.541 and 0.103 whilst the values returned by the model are 0.559, 0.366 and 0.400 so the first set of values are very close however the next two sets are not.

Test model using random rows of validation data

In [144]:
np.random.seed([4])

shuffle_fog_test.sample(n = 3)

,dayofweek,fog,num_accidents
2131,6,0,0.665919
2347,5,0,0.669643
2200,5,1,0.354911


In [145]:
input = pd.DataFrame.from_dict(data = 
				{'dayofweek' : [5,7,4],
         'fog' : [0, 0, 0]
        })
					

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_f', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
predslistscale = preds['scores']*850
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4f9aa7a610>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_f', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_f/mode

It can be seen that the actual values in the test data are 0.635, 0.667 and 0.587.  The values predicted by the model though are 0.519, 0.599 and 0.479.  This is not too far off though not extremely accurate either.

### Create and test model for fog and temperature

In [146]:
#shuffle fog/temperature dataset 
shuffle_fogtemp = df_tempfog_numacc.iloc[np.random.permutation(len(df_tempfog_numacc))]

#create training and validation data
shuffle_fogtemp_train, shuffle_fogtemp_test = train_test_split(shuffle_fogtemp, test_size=0.1)

#select variables used for prediction
predictors_fogtemp = shuffle_fogtemp_train.iloc[:,0:3]

print(predictors_fogtemp[:10])

      dayofweek  temp  fog
1364          2  57.6    0
1562          4  49.8    0
968           5  73.8    0
1875          2  39.4    0
37            5  34.8    0
158           7  56.1    0
1790          1  50.1    0
1806          3  38.4    0
509           1  54.6    0
626           6  61.1    0


In [147]:
shuffle_fogtemp_train[:5]

,dayofweek,temp,fog,num_accidents
1364,2,57.6,0,0.444206
1562,4,49.8,0,0.312236
968,5,73.8,0,0.526427
1875,2,39.4,0,0.163677
37,5,34.8,0,0.483945


In [148]:
#create targets 
targets_fogtemp = shuffle_fogtemp_train.iloc[:,3]

print(targets_fogtemp[:10])

1364    0.444206
1562    0.312236
968     0.526427
1875    0.163677
37      0.483945
158     1.000000
1790    0.179325
1806    0.746835
509     0.291667
626     0.671296
Name: num_accidents, dtype: float64


Create model

In [149]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

shutil.rmtree('/tmp/linear_regression_trained_model_ft', ignore_errors=True)

estimator_fogtemp = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_ft', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors_fogtemp.values)))

# Starting to train
print("starting to train");

# Train the model. 
estimator_fogtemp.fit(predictors_fogtemp[:trainsize].values, targets_fogtemp[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_ACCIDENTS, steps=10000)

# Check  predictions 
preds = estimator_fogtemp.predict(x=predictors_fogtemp[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_ACCIDENTS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE
rmse = np.sqrt(np.mean((targets_fogtemp[trainsize:].values - predslistscale)**2))
print('LinearRegression Fog and Temperature has RMSE of {0}'.format(rmse));

# Calculate the mean of the Number of Accidents.
avg = np.mean(shuffle_fogtemp_train['num_accidents'][:trainsize])

# Calculate the RMSE 
rmse = np.sqrt(np.mean((shuffle_fogtemp_train['num_accidents'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4fa086b110>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_ft', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Runnin

The RMSE values are 0.222 and 0.194 which are good so the model will now be tested using the validation dataset.

Test model using first three rows of the test data

In [150]:
shuffle_fogtemp_test[:3]

,dayofweek,temp,fog,num_accidents
942,7,76.7,0,0.837209
1160,1,34.8,0,0.506438
62,2,37.4,0,0.337156


In [151]:
input = pd.DataFrame.from_dict(data = 
				{'dayofweek' : [7,1,2],
         'temp' : [76.7, 34.8, 37.4],
     'fog' : [0, 0, 0]
        })
					

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_ft', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
predslistscale = preds['scores']*850
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4f9b333110>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_ft', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_ft/mo

The normalised number of accidents from the test data are 0.837, 0.506 and 0.337 however the predictions made by the model are 0.538, 0.206 and 0.262.  The first two sets of values are not accurate at all though the third is a bit closer.

Test model using random rows from the test data

In [152]:
np.random.seed([5])

shuffle_fogtemp_test.sample(n = 3)

,dayofweek,temp,fog,num_accidents
2466,5,67.6,0,0.743304
2125,7,39.8,0,0.780269
749,3,41.7,0,0.293869


In [153]:
input = pd.DataFrame.from_dict(data = 
				{'dayofweek' : [5,7,3],
         'temp' : [67.6, 39.8, 41.7],
     'fog' : [0, 0, 0]
        })
					

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_ft', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
predslistscale = preds['scores']*850
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4fa0770c50>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_ft', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_ft/mo

The actual normalised values of number of accidents tested are 0.743, 0.780 and 0.294 whilst the predictions of the model are 0.427, 0.541 and 0.317.  This gives a similar result to the previous test in that the first two sets of values are not close at all but the third one is ok.

### Create and test model for temperature, visibility and fog

In [154]:
#shuffle fog/temperature/visibility dataset 
shuffle_fogtempvis = df_tempfogvisib_numacc.iloc[np.random.permutation(len(df_tempfogvisib_numacc))]

#create training and validation data
shuffle_fogtempvis_train, shuffle_fogtempvis_test = train_test_split(shuffle_fogtempvis, test_size=0.1)

#select variables used for prediction
predictors_fogtempvis = shuffle_fogtempvis_train.iloc[:,0:4]

print(predictors_fogtempvis[:10])

      dayofweek  temp  visib  fog
2080          4  68.9    7.4    1
353           6  39.1   10.0    0
1127          3  47.6    6.9    0
630           3  67.9    8.2    1
2066          4  73.0    3.5    1
1732          6  69.4    4.9    1
42            3  36.4    7.6    0
1396          6  46.5    9.8    0
1622          1  60.4    9.3    0
1744          4  68.8    5.9    1


In [155]:
shuffle_fogtempvis_train[:5]

,dayofweek,temp,visib,fog,num_accidents
2080,4,68.9,7.4,1,0.325112
353,6,39.1,10.0,0,0.577982
1127,3,47.6,6.9,0,0.753219
630,3,67.9,8.2,1,0.699074
2066,4,73.0,3.5,1,0.708520


In [156]:
#create targets 
targets_fogtempvis = shuffle_fogtempvis_train.iloc[:,4]

print(targets_fogtempvis[:10])

2080    0.325112
353     0.577982
1127    0.753219
630     0.699074
2066    0.708520
1732    0.778481
42      0.275229
1396    1.000000
1622    0.620253
1744    0.837553
Name: num_accidents, dtype: float64


Create model

In [157]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

shutil.rmtree('/tmp/linear_regression_trained_model_ftv', ignore_errors=True)

estimator_fogtempvis = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_ftv', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors_fogtempvis.values)))

# Starting to train
print("starting to train");

# Train the model. 
estimator_fogtempvis.fit(predictors_fogtempvis[:trainsize].values, targets_fogtempvis[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_ACCIDENTS, steps=10000)

# Check  predictions 
preds = estimator_fogtempvis.predict(x=predictors_fogtempvis[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_ACCIDENTS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE
rmse = np.sqrt(np.mean((targets_fogtempvis[trainsize:].values - predslistscale)**2))
print('LinearRegression Fog, Temperature and Visibility has RMSE of {0}'.format(rmse));

# Calculate the mean of the Number of Accidents.
avg = np.mean(shuffle_fogtempvis_train['num_accidents'][:trainsize])

# Calculate the RMSE 
rmse = np.sqrt(np.mean((shuffle_fogtempvis_train['num_accidents'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4fa0bba4d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_ftv', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Runni

The RMSE values of 0.381 and 0.205 are not particularly close for this model but it will be tested with the validation data.

Test model using first three rows of validation data

In [158]:
shuffle_fogtempvis_test[:3]

,dayofweek,temp,visib,fog,num_accidents
1137,6,31.5,9.5,0,0.551502
1510,1,37.0,9.8,0,0.419831
1230,1,59.4,6.9,1,0.480687


In [159]:
input = pd.DataFrame.from_dict(data = 
				{'dayofweek' : [6,1,1],
         'temp' : [31.5, 37.0, 59.4],
     'visib' : [9.5, 9.8, 6.9],
     'fog' : [0, 0, 1]
     
        })
					

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_ftv', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
predslistscale = preds['scores']*850
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4f9b96ae50>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_ftv', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_ftv/

It can be seen that the actual values for normalised number of accidents is 0.552, 0.420 and 0.481.  The model however predicted results of 0.790, 0.538 and 0.748 showing little accuracy.

Test model using random rows

In [160]:
np.random.seed([6])

shuffle_fogtempvis_test.sample(n = 3)

,dayofweek,temp,visib,fog,num_accidents
850,6,48.1,10.0,0,0.598309
2257,6,22.1,10.0,0,0.448661
1458,5,39.1,10.0,0,0.388412


In [161]:
input = pd.DataFrame.from_dict(data = 
				{'dayofweek' : [6,6,5],
         'temp' : [48.1, 22.1, 39.1],
     'visib' : [10, 10, 10],
     'fog' : [0, 0, 0]
        })
					

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_ftv', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
predslistscale = preds['scores']*850
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4fa2f547d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_ftv', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_ftv/

The normalised number of accident values from the test data are 0.598, 0.449 and 0.388 but the model returned values of 0.928, 0.716 and 0.795, again showing little accuracy and supporting the previous test and the RMSE test that the model is not accurate.

## DNN Models

### Create dataframes from data created previously for dnn models

In [162]:
df_dnn_numacc = pd.read_csv('https://raw.githubusercontent.com/078373uhi/078373uhi-DAOTW_A2/main/dnndata_numacc.csv', index_col=0, )
df_dnn_fog = pd.read_csv('https://raw.githubusercontent.com/078373uhi/078373uhi-DAOTW_A2/main/dnndata_fog_numacc.csv', index_col=0, )
df_dnn_temp = pd.read_csv('https://raw.githubusercontent.com/078373uhi/078373uhi-DAOTW_A2/main/dnndata_temp_numacc.csv', index_col=0, )
df_dnn_visib = pd.read_csv('https://raw.githubusercontent.com/078373uhi/078373uhi-DAOTW_A2/main/dnndata_visib_numacc.csv', index_col=0, )
df_dnn_fogvisib = pd.read_csv('https://raw.githubusercontent.com/078373uhi/078373uhi-DAOTW_A2/main/dnndata_fogvisib_numacc.csv', index_col=0, )
df_dnn_tempfog = pd.read_csv('https://raw.githubusercontent.com/078373uhi/078373uhi-DAOTW_A2/main/dnndata_tempfog_numacc.csv', index_col=0, )
df_dnn_tempvisib = pd.read_csv('https://raw.githubusercontent.com/078373uhi/078373uhi-DAOTW_A2/main/dnndata_tempvisib_numacc.csv', index_col=0, )
df_dnn_tempfogvisib = pd.read_csv('https://raw.githubusercontent.com/078373uhi/078373uhi-DAOTW_A2/main/dnndata_tempfogvisib_numacc.csv', index_col=0, )

**Check dataframes**

In [163]:
print(df_dnn_numacc[:10])
print(df_dnn_fog[:10])
print(df_dnn_temp[:10])
print(df_dnn_visib[:10])
print(df_dnn_fogvisib[:10])
print(df_dnn_tempfog[:10])
print(df_dnn_tempvisib[:10])
print(df_dnn_tempfogvisib[:10])

    Apr  Aug  Dec  Feb  Jan  Jul  ...  Sun  Thu  Tue  Wed  year  number_of_accidents
1     0    0    0    0    1    0  ...    0    0    1    0  2013             0.059633
2     0    0    0    0    1    0  ...    0    0    0    1  2013             0.286697
3     0    0    0    0    1    0  ...    0    1    0    0  2013             0.444954
4     0    0    0    0    1    0  ...    0    0    0    0  2013             0.344037
5     0    0    0    0    1    0  ...    0    0    0    0  2013             0.077982
6     0    0    0    0    1    0  ...    1    0    0    0  2013             0.087156
7     0    0    0    0    1    0  ...    0    0    0    0  2013             0.380734
8     0    0    0    0    1    0  ...    0    0    1    0  2013             0.362385
9     0    0    0    0    1    0  ...    0    0    0    1  2013             0.302752
10    0    0    0    0    1    0  ...    0    1    0    0  2013             0.458716

[10 rows x 21 columns]
    Apr  Aug  Dec  Feb  Jan  Jul  ...  Th

### Create and test model for visiblity

In [164]:
# shuffle visibility dataset
shuffle_visib = df_dnn_visib.iloc[np.random.permutation(len(df_dnn_visib))]

#create training and validation data
shuffle_visib_train, shuffle_visib_test = train_test_split(shuffle_visib, test_size=0.1)

#check training set
print(shuffle_visib_train[:5])

# choose predictors
predictors_visib = shuffle_visib_train.iloc[:,0:21]

# check predictors.
print(predictors_visib[:6])

      Apr  Aug  Dec  Feb  Jan  ...  Tue  Wed  year  visib  number_of_accidents
1735    0    0    0    0    0  ...    1    0  2017    8.7             0.662447
1473    0    0    0    1    0  ...    0    1  2017    9.7             0.428270
2152    0    0    1    0    0  ...    0    0  2018    5.3             0.858744
2174    0    0    0    0    1  ...    0    0  2019   10.0             0.042411
2372    0    0    0    0    0  ...    1    0  2019    6.6             0.631696

[5 rows x 22 columns]
      Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  Sat  Sun  Thu  Tue  Wed  year  visib
1735    0    0    0    0    0    0    0  ...    0    0    0    1    0  2017    8.7
1473    0    0    0    1    0    0    0  ...    0    0    0    0    1  2017    9.7
2152    0    0    1    0    0    0    0  ...    0    0    0    0    0  2018    5.3
2174    0    0    0    0    1    0    0  ...    0    1    0    0    0  2019   10.0
2372    0    0    0    0    0    1    0  ...    0    0    0    1    0  2019    6.6
46   

In [165]:
# Create outcome data - targets
targets_visib = shuffle_visib_train.iloc[:,21]

# check targets data.
print(targets_visib[:6])

1735    0.662447
1473    0.428270
2152    0.858744
2174    0.042411
2372    0.631696
46      0.571101
Name: number_of_accidents, dtype: float64


In [166]:
SCALE_num_accidents = 1.0

In [167]:
# Split our data into a training set = 80% of the length of the shuffle array
trainsize = int(len(shuffle_visib_train['number_of_accidents'])*0.8)
# The test set size = 20% of the length of the shuffle array.
testsize = len(shuffle_visib_train['number_of_accidents']) - trainsize

# Define the number of predictors
nppredictors = 21
# Define the number of targets
noutputs = 1

Create model

In [168]:
# create model
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes previous saved model
shutil.rmtree('/tmp/DNN_regression_trained_model_v', ignore_errors=True)

estimator_visib = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_v', hidden_units=[20,18,14], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors_visib.values)))

# show model is starting to train
print("starting to train");

# Train the model
estimator_visib.fit(predictors_visib[:trainsize].values, targets_visib[:trainsize].values.reshape(trainsize, noutputs)/SCALE_num_accidents, steps=10000)

# check predictions
preds = estimator_visib.predict(x=predictors_visib[trainsize:].values)

# Apply the Scale value
predslistscale = preds['scores']*SCALE_num_accidents

# Calculate RMSE 
rmse = np.sqrt(np.mean((targets_visib[trainsize:].values - predslistscale)**2))
print('DNNRegression visibility has RMSE of {0}'.format(rmse));

# Calculate the mean of the number of accident values.
avg = np.mean(shuffle_visib_train['number_of_accidents'][:trainsize])

# Calculate the RMSE using number of accident values and the mean of all target values.
rmse = np.sqrt(np.mean((shuffle_visib_train['number_of_accidents'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4f9b96a350>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_v', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running lo

The RMSE results are 0.146 and 0.197 which are close so the model will be tested on the validation data.

Test model on first three rows of validation data.

In [169]:
shuffle_visib_test[:3]

,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,Fri,Mon,Sat,Sun,Thu,Tue,Wed,year,visib,number_of_accidents
234,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2013,7.8,0.644495
1542,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2017,8.8,0.312236
588,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2014,9.8,0.543981


In [170]:
# create test using data above
input = pd.DataFrame.from_dict(data = 
				{
         'Apr' : [0,1,0],
         'Aug' : [1,0,1],
         'Dec' : [0,0,0],
         'Feb' : [0,0,0],
         'Jan' : [0,0,0],
         'Jul' : [0,0,0],
         'Jun' : [0,0,0],
         'Mar' : [0,0,0],
         'May' : [0,0,0],
         'Nov' : [0,0,0],
         'Oct' : [0,0,0],
         'Sep' : [0,0,0],
         'Fri' : [1,0,0],
         'Mon' : [0,0,1],
         'Sat' : [0,0,0],
         'Sun' : [0,0,0],
         'Thu' : [0,0,0],
         'Tue' : [0,1,0],
         'Wed' : [0,0,0],
         'year' : [2013,2017,2014],
         'visib' : [7.8, 8.8, 9.8]
        })

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_v', hidden_units=[20,18,14], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
prednorm = format(str(predslistnorm))
print(prednorm)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4fa0bd44d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_v', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_v/model.ckpt

The normalised number of accidents for the test data is 0.644, 0.312 and 0.544 and the model predicted 0.661, 0.553 and 0.511 so the first and third sets of values were very accurate though the second one was less so.

Test model on random data selected from validation data

In [171]:
np.random.seed([7])

shuffle_visib_test.sample(n = 3)

,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,Fri,Mon,Sat,Sun,Thu,Tue,Wed,year,visib,number_of_accidents
261,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,2013,10.0,0.415138
1068,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2015,2.4,0.866808
1944,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,2018,8.0,0.896861


In [172]:
# create test using data above
input = pd.DataFrame.from_dict(data = 
				{
         'Apr' : [0,0,0],
         'Aug' : [0,0,0],
         'Dec' : [0,1,0],
         'Feb' : [0,0,0],
         'Jan' : [0,0,0],
         'Jul' : [0,0,0],
         'Jun' : [0,0,0],
         'Mar' : [0,0,0],
         'May' : [0,0,1],
         'Nov' : [0,0,0],
         'Oct' : [0,0,0],
         'Sep' : [1,0,0],
         'Fri' : [0,1,1],
         'Mon' : [0,0,0],
         'Sat' : [0,0,0],
         'Sun' : [0,0,0],
         'Thu' : [1,0,0],
         'Tue' : [0,0,0],
         'Wed' : [0,0,0],
         'year' : [2013,2015,2018],
         'visib' : [10, 2.4, 8]
        })

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_v', hidden_units=[20,18,14], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
prednorm = format(str(predslistnorm))
print(prednorm)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4fa0a44590>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_v', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_v/model.ckpt

The normalised number of accidents for the random test data sample is 0.415, 0.869 and 0.897 and the model predicted 0.616, 0.692 and 0.726 so this is much less accurate than the previous test.

### Create and test model for temperature

In [173]:
# shuffle temperature dataset
shuffle_temp = df_dnn_temp.iloc[np.random.permutation(len(df_dnn_temp))]

#create training and validation data
shuffle_temp_train, shuffle_temp_test = train_test_split(shuffle_temp, test_size=0.1)

#check training set
print(shuffle_temp_train[:5])

# choose predictors
predictors_temp = shuffle_temp_train.iloc[:,0:21]

# check predictors.
print(predictors_temp[:6])

      Apr  Aug  Dec  Feb  Jan  ...  Tue  Wed  year  temp  number_of_accidents
1369    0    0    0    0    0  ...    1    0  2016  63.8             0.482833
824     1    0    0    0    0  ...    0    0  2015  47.5             0.680761
322     0    0    0    0    0  ...    1    0  2013  47.0             0.561927
1780    0    0    1    0    0  ...    0    0  2017  41.0             0.656118
659     0    0    0    0    0  ...    1    0  2014  54.3             0.488426

[5 rows x 22 columns]
      Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  Sat  Sun  Thu  Tue  Wed  year  temp
1369    0    0    0    0    0    0    0  ...    0    0    0    1    0  2016  63.8
824     1    0    0    0    0    0    0  ...    0    0    0    0    0  2015  47.5
322     0    0    0    0    0    0    0  ...    0    0    0    1    0  2013  47.0
1780    0    0    1    0    0    0    0  ...    0    0    0    0    0  2017  41.0
659     0    0    0    0    0    0    0  ...    0    0    0    1    0  2014  54.3
1051    0    0   

In [174]:
# Create outcome data - targets
targets_temp = shuffle_temp_train.iloc[:,21]

# check targets data.
print(targets_temp[:6])

1369    0.482833
824     0.680761
322     0.561927
1780    0.656118
659     0.488426
1051    0.822410
Name: number_of_accidents, dtype: float64


In [175]:
SCALE_num_accidents = 1.0

In [176]:
# Split our data into a training set = 80% of the length of the shuffle array
trainsize = int(len(shuffle_temp_train['number_of_accidents'])*0.8)
# The test set size = 20% of the length of the shuffle array.
testsize = len(shuffle_temp_train['number_of_accidents']) - trainsize

# Define the number of predictors
nppredictors = 21
# Define the number of targets
noutputs = 1

Create model

In [101]:
# create model
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes previous saved model
shutil.rmtree('/tmp/DNN_regression_trained_model_t', ignore_errors=True)

estimator_temp = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_t', hidden_units=[20,18,14], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors_temp.values)))

# show model is starting to train
print("starting to train");

# Train the model
estimator_temp.fit(predictors_temp[:trainsize].values, targets_temp[:trainsize].values.reshape(trainsize, noutputs)/SCALE_num_accidents, steps=10000)

# check predictions
preds = estimator_temp.predict(x=predictors_temp[trainsize:].values)

# Apply the Scale value
predslistscale = preds['scores']*SCALE_num_accidents

# Calculate RMSE 
rmse = np.sqrt(np.mean((targets_temp[trainsize:].values - predslistscale)**2))
print('DNNRegression temperature has RMSE of {0}'.format(rmse));

# Calculate the mean of the number of accident values.
avg = np.mean(shuffle_temp_train['number_of_accidents'][:trainsize])

# Calculate the RMSE using number of accident values and the mean of all target values.
rmse = np.sqrt(np.mean((shuffle_temp_train['number_of_accidents'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4fa0b0a210>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_t', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running lo

The RMSE values are 0.288 and 0.197 which is not hugely accurate but the model will tested with the validation data nonetheless.

Test data with first three rows of validation data.

In [102]:
shuffle_temp_test[:3]

,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,Fri,Mon,Sat,Sun,Thu,Tue,Wed,year,temp,number_of_accidents
469,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2014,43.1,0.372685
225,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2013,68.4,0.612385
1757,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,2017,43.9,0.721519


In [103]:
# create test using data above
input = pd.DataFrame.from_dict(data = 
				{
         'Apr' : [1,0,0],
         'Aug' : [0,1,0],
         'Dec' : [0,0,0],
         'Feb' : [0,0,0],
         'Jan' : [0,0,0],
         'Jul' : [0,0,0],
         'Jun' : [0,0,0],
         'Mar' : [0,0,0],
         'May' : [0,0,0],
         'Nov' : [0,0,1],
         'Oct' : [0,0,0],
         'Sep' : [0,0,0],
         'Fri' : [0,0,0],
         'Mon' : [1,0,0],
         'Sat' : [0,0,0],
         'Sun' : [0,0,0],
         'Thu' : [0,0,0],
         'Tue' : [0,1,0],
         'Wed' : [0,0,1],
         'year' : [2014,2013,2017],
         'temp' : [43.1, 68.4, 43.9]
        })

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_t', hidden_units=[20,18,14], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
prednorm = format(str(predslistnorm))
print(prednorm)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4fa09cfa50>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_t', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_t/model.ckpt

The number of accidents from the test data are 0.373, 0.612 and 0.722 and the model returned values of 0.597, -0.081 and 0.585 so these are not accurate at all.

Test with randomly selected rows from the validation data.

In [104]:
np.random.seed([8])

shuffle_temp_test.sample(n = 3)

,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,Fri,Mon,Sat,Sun,Thu,Tue,Wed,year,temp,number_of_accidents
188,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,2013,72.6,0.600917
2482,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,2019,41.5,0.198661
1677,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2017,62.3,0.267932


In [105]:
# create test using data above
input = pd.DataFrame.from_dict(data = 
				{
         'Apr' : [0,0,0],
         'Aug' : [0,0,1],
         'Dec' : [0,0,0],
         'Feb' : [0,0,0],
         'Jan' : [0,0,0],
         'Jul' : [1,0,0],
         'Jun' : [0,0,0],
         'Mar' : [0,0,0],
         'May' : [0,0,0],
         'Nov' : [0,1,0],
         'Oct' : [0,0,0],
         'Sep' : [0,0,0],
         'Fri' : [0,0,0],
         'Mon' : [1,0,0],
         'Sat' : [0,0,0],
         'Sun' : [0,1,1],
         'Thu' : [0,0,0],
         'Tue' : [0,0,0],
         'Wed' : [0,0,1],
         'year' : [2013,2019,2017],
         'temp' : [72.6, 41.5, 62.3]
        })

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_t', hidden_units=[20,18,14], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
prednorm = format(str(predslistnorm))
print(prednorm)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4fa0763cd0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_t', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_t/model.ckpt

The number of accidents from the test data are 0.601, 0.199 and 0.268 and the model returned values of 0.493, 0.386 and 0.447 so these are not accurate at all and reflect the result of the previous test.

### Create and test DNN model for fog

In [106]:
# shuffle temperature dataset
shuffle_fog = df_dnn_fog.iloc[np.random.permutation(len(df_dnn_fog))]

#create training and validation data
shuffle_fog_train, shuffle_fog_test = train_test_split(shuffle_fog, test_size=0.1)

#check training set
print(shuffle_fog_train[:5])

# choose predictors
predictors_fog = shuffle_fog_train.iloc[:,0:21]

# check predictors.
print(predictors_fog[:6])

      Apr  Aug  Dec  Feb  Jan  ...  Tue  Wed  year  fog  number_of_accidents
2298    0    0    0    0    0  ...    0    0  2019    0             0.828125
43      0    0    0    1    0  ...    1    0  2013    1             0.327982
412     0    0    0    1    0  ...    0    0  2014    1             0.525463
745     0    0    0    0    1  ...    0    0  2015    0             0.268499
807     0    0    0    0    0  ...    1    0  2015    0             0.572939

[5 rows x 22 columns]
      Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  Sat  Sun  Thu  Tue  Wed  year  fog
2298    0    0    0    0    0    0    0  ...    0    0    0    0    0  2019    0
43      0    0    0    1    0    0    0  ...    0    0    0    1    0  2013    1
412     0    0    0    1    0    0    0  ...    1    0    0    0    0  2014    1
745     0    0    0    0    1    0    0  ...    0    0    0    0    0  2015    0
807     0    0    0    0    0    0    0  ...    0    0    0    1    0  2015    0
2126    0    0    0    0    0

In [107]:
# Create outcome data - targets
targets_fog = shuffle_fog_train.iloc[:,21]

# check targets data.
print(targets_fog[:6])

2298    0.828125
43      0.327982
412     0.525463
745     0.268499
807     0.572939
2126    0.188341
Name: number_of_accidents, dtype: float64


In [108]:
SCALE_num_accidents = 1.0

In [109]:
# Split our data into a training set = 80% of the length of the shuffle array
trainsize = int(len(shuffle_fog_train['number_of_accidents'])*0.8)
# The test set size = 20% of the length of the shuffle array.
testsize = len(shuffle_fog_train['number_of_accidents']) - trainsize

# Define the number of predictors
nppredictors = 21
# Define the number of targets
noutputs = 1

Create model

In [110]:
# create model
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes previous saved model
shutil.rmtree('/tmp/DNN_regression_trained_model_f', ignore_errors=True)

estimator_fog = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_f', hidden_units=[20,18,14], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors_fog.values)))

# show model is starting to train
print("starting to train");

# Train the model
estimator_fog.fit(predictors_fog[:trainsize].values, targets_fog[:trainsize].values.reshape(trainsize, noutputs)/SCALE_num_accidents, steps=10000)

# check predictions
preds = estimator_fog.predict(x=predictors_fog[trainsize:].values)

# Apply the Scale value
predslistscale = preds['scores']*SCALE_num_accidents

# Calculate RMSE 
rmse = np.sqrt(np.mean((targets_fog[trainsize:].values - predslistscale)**2))
print('DNNRegression fog has RMSE of {0}'.format(rmse));

# Calculate the mean of the number of accident values.
avg = np.mean(shuffle_fog_train['number_of_accidents'][:trainsize])

# Calculate the RMSE using number of accident values and the mean of all target values.
rmse = np.sqrt(np.mean((shuffle_fog_train['number_of_accidents'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4f9ae0e610>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_f', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running lo

The RMSE values are 0.167 and 0.210 which is fairly close so this model will now be tested with the validation data.

Test data with first three rows of validation data

In [111]:
shuffle_fog_test[:3]

,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,Fri,Mon,Sat,Sun,Thu,Tue,Wed,year,fog,number_of_accidents
32,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2013,0,0.502294
2418,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,2019,0,0.296875
923,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,2015,0,0.198732


In [112]:
# create test using data above
input = pd.DataFrame.from_dict(data = 
				{
         'Apr' : [0,0,0],
         'Aug' : [0,0,0],
         'Dec' : [0,0,0],
         'Feb' : [1,0,0],
         'Jan' : [0,0,0],
         'Jul' : [0,0,1],
         'Jun' : [0,0,0],
         'Mar' : [0,0,0],
         'May' : [0,0,0],
         'Nov' : [0,0,0],
         'Oct' : [0,0,0],
         'Sep' : [0,1,0],
         'Fri' : [0,0,0],
         'Mon' : [1,0,0],
         'Sat' : [0,1,1],
         'Sun' : [0,0,0],
         'Thu' : [0,0,0],
         'Tue' : [0,0,0],
         'Wed' : [0,0,0],
         'year' : [2013,2019,2015],
         'fog' : [0,0,0]
        })

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_f', hidden_units=[20,18,14], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
prednorm = format(str(predslistnorm))
print(prednorm)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4f9a8e4ad0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_f', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_f/model.ckpt

The actual values from the test data are 0.502, 0.297 and 0.199 while the predicted values from the model are 0.463, 0.498 and 0.464.  The first pair of values are quite close but the second and third pairs are not.

Test model with random three rows from validation data.

In [113]:
np.random.seed([9])

shuffle_fog_test.sample(n = 3)

,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,Fri,Mon,Sat,Sun,Thu,Tue,Wed,year,fog,number_of_accidents
28,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2013,0,0.348624
2476,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,2019,0,0.281250
1076,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2015,1,0.340381


In [114]:
# create test using data above
input = pd.DataFrame.from_dict(data = 
				{
         'Apr' : [0,0,0],
         'Aug' : [0,0,0],
         'Dec' : [0,0,1],
         'Feb' : [0,0,0],
         'Jan' : [1,0,0],
         'Jul' : [0,0,0],
         'Jun' : [0,0,0],
         'Mar' : [0,0,0],
         'May' : [0,0,0],
         'Nov' : [0,1,0],
         'Oct' : [0,0,0],
         'Sep' : [0,0,0],
         'Fri' : [0,0,0],
         'Mon' : [1,1,0],
         'Sat' : [0,1,0],
         'Sun' : [0,0,1],
         'Thu' : [0,0,0],
         'Tue' : [0,0,0],
         'Wed' : [0,0,0],
         'year' : [2013,2019,2015],
         'fog' : [0,0,1]
        })

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_f', hidden_units=[20,18,14], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
prednorm = format(str(predslistnorm))
print(prednorm)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4f9aac09d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_f', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_f/model.ckpt

The actual values from the validation data are 0.349, 0.281 and 0.340 but the results from the model are 0.460, 0.486 and 0.358.  The third row tested was very close though the other two were not.

### Create and test model for temperature and visibility

In [179]:
# shuffle temperature and visibility dataset
shuffle_tempvis = df_dnn_tempvisib.iloc[np.random.permutation(len(df_dnn_tempvisib))]

#create training and validation data
shuffle_tempvis_train, shuffle_tempvis_test = train_test_split(shuffle_tempvis, test_size=0.1)

#check training set
print(shuffle_tempvis_train[:5])

# choose predictors
predictors_tempvis = shuffle_tempvis_train.iloc[:,0:22]

# check predictors.
print(predictors_tempvis[:6])

      Apr  Aug  Dec  Feb  Jan  ...  Wed  year  temp  visib  number_of_accidents
2241    0    0    0    0    0  ...    0  2019  40.7   10.0             0.600446
1132    0    0    0    1    0  ...    0  2016  36.5   10.0             0.553648
1730    0    0    0    0    0  ...    0  2017  59.7   10.0             0.833333
305     0    0    0    0    0  ...    0  2013  57.4    6.4             0.449541
2153    0    0    1    0    0  ...    0  2018  48.3    4.1             0.378924

[5 rows x 23 columns]
      Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  Sun  Thu  Tue  Wed  year  temp  visib
2241    0    0    0    0    0    0    0  ...    0    1    0    0  2019  40.7   10.0
1132    0    0    0    1    0    0    0  ...    0    0    1    0  2016  36.5   10.0
1730    0    0    0    0    0    0    0  ...    0    1    0    0  2017  59.7   10.0
305     0    0    0    0    0    0    0  ...    0    0    0    0  2013  57.4    6.4
2153    0    0    1    0    0    0    0  ...    0    0    0    0  2018  48.3 

In [181]:
# Create outcome data - targets
targets_tempvis = shuffle_tempvis_train.iloc[:,22]

# check targets data.
print(targets_tempvis[:6])

2241    0.600446
1132    0.553648
1730    0.833333
305     0.449541
2153    0.378924
2176    0.587054
Name: number_of_accidents, dtype: float64


In [182]:
SCALE_num_accidents = 1.0

In [183]:
# Split our data into a training set = 80% of the length of the shuffle array
trainsize = int(len(shuffle_tempvis_train['number_of_accidents'])*0.8)
# The test set size = 20% of the length of the shuffle array.
testsize = len(shuffle_tempvis_train['number_of_accidents']) - trainsize

# Define the number of predictors
nppredictors = 22
# Define the number of targets
noutputs = 1

Create model

In [184]:
# create model
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes previous saved model
shutil.rmtree('/tmp/DNN_regression_trained_model_tv', ignore_errors=True)

estimator_tempvis = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_tv', hidden_units=[20,18,14], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors_tempvis.values)))

# show model is starting to train
print("starting to train");

# Train the model
estimator_tempvis.fit(predictors_tempvis[:trainsize].values, targets_tempvis[:trainsize].values.reshape(trainsize, noutputs)/SCALE_num_accidents, steps=10000)

# check predictions
preds = estimator_tempvis.predict(x=predictors_tempvis[trainsize:].values)

# Apply the Scale value
predslistscale = preds['scores']*SCALE_num_accidents

# Calculate RMSE 
rmse = np.sqrt(np.mean((targets_tempvis[trainsize:].values - predslistscale)**2))
print('DNNRegression temperature and visibility has RMSE of {0}'.format(rmse));

# Calculate the mean of the number of accident values.
avg = np.mean(shuffle_tempvis_train['number_of_accidents'][:trainsize])

# Calculate the RMSE using number of accident values and the mean of all target values.
rmse = np.sqrt(np.mean((shuffle_tempvis_train['number_of_accidents'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4f9a92e310>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_tv', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running l

The RMSE values are 0.147 and 0.196 which are close so the model will now be tested with the validation data.

Test the model with the first three rows of validation data

In [185]:
shuffle_tempvis_test[:3]

,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,Fri,Mon,Sat,Sun,Thu,Tue,Wed,year,temp,visib,number_of_accidents
940,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2015,67.9,7.5,0.515856
1766,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,2017,40.8,10.0,0.276371
2183,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2019,15.8,10.0,0.607143


In [187]:
# create test using data above
input = pd.DataFrame.from_dict(data = 
				{
         'Apr' : [0,0,0],
         'Aug' : [1,0,0],
         'Dec' : [0,0,0],
         'Feb' : [0,0,0],
         'Jan' : [0,0,1],
         'Jul' : [0,0,0],
         'Jun' : [0,0,0],
         'Mar' : [0,0,0],
         'May' : [0,0,0],
         'Nov' : [0,1,0],
         'Oct' : [0,0,0],
         'Sep' : [0,0,0],
         'Fri' : [0,0,0],
         'Mon' : [0,0,0],
         'Sat' : [0,0,0],
         'Sun' : [0,0,0],
         'Thu' : [0,0,0],
         'Tue' : [1,0,1],
         'Wed' : [0,0,0],
         'year' : [2015,2017,2019],
         'temp' : [67.9,40.8,15.8],
     'visib' : [7.5,10,10]
        })

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_tv', hidden_units=[20,18,14], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
prednorm = format(str(predslistnorm))
print(prednorm)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4f9b2c6e50>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_tv', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_tv/model.ck

The actual accident values from the test data are 0.516, 0.276 and 0.607 whilst the values returned by the model are 0.541, 0.527 and 0.426.  So the first pair are close but the other two are not.

Test the model using three random rows from the validation model

In [188]:
np.random.seed([10])

shuffle_tempvis_test.sample(n = 3)

,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,Fri,Mon,Sat,Sun,Thu,Tue,Wed,year,temp,visib,number_of_accidents
1397,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,2016,54.4,8.0,0.729614
912,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,2015,67.4,9.8,0.634249
1772,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,2017,43.2,10.0,0.616034


In [189]:
# create test using data above
input = pd.DataFrame.from_dict(data = 
				{
         'Apr' : [0,0,0],
         'Aug' : [0,0,0],
         'Dec' : [0,0,0],
         'Feb' : [0,0,0],
         'Jan' : [0,0,0],
         'Jul' : [0,1,0],
         'Jun' : [0,0,0],
         'Mar' : [0,0,0],
         'May' : [0,0,0],
         'Nov' : [1,0,1],
         'Oct' : [0,0,0],
         'Sep' : [0,0,0],
         'Fri' : [0,0,0],
         'Mon' : [0,0,0],
         'Sat' : [0,0,0],
         'Sun' : [0,0,0],
         'Thu' : [0,0,1],
         'Tue' : [0,1,0],
         'Wed' : [1,0,0],
         'year' : [2016,2015,2017],
         'temp' : [54.4,67.4,43.2],
     'visib' : [8,9.8,10]
        })

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_tv', hidden_units=[20,18,14], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
prednorm = format(str(predslistnorm))
print(prednorm)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4f9a88bf10>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_tv', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_tv/model.ck

The number of accidents from the test data is 0.730, 0.634 and 0.616 and the model predicted values of 0.580, 0.559 and 0.583 so the first one is quite far off but the other two are fairly close showing better accuracy than the previous test.

### Create and test a model for fog and visibility




In [190]:
# shuffle fog and visibility dataset
shuffle_fogvis = df_dnn_fogvisib.iloc[np.random.permutation(len(df_dnn_fogvisib))]

#create training and validation data
shuffle_fogvis_train, shuffle_fogvis_test = train_test_split(shuffle_fogvis, test_size=0.1)

#check training set
print(shuffle_fogvis_train[:5])

# choose predictors
predictors_fogvis = shuffle_fogvis_train.iloc[:,0:22]

# check predictors.
print(predictors_fogvis[:6])

      Apr  Aug  Dec  Feb  Jan  ...  Wed  year  fog  visib  number_of_accidents
1348    0    0    0    0    0  ...    0  2016    0   10.0             0.553648
2409    0    0    0    0    0  ...    0  2019    0    9.5             0.609375
462     1    0    0    0    0  ...    0  2014    0    7.0             0.356481
2341    0    0    0    0    0  ...    0  2019    1    7.9             0.680804
1738    0    0    0    0    0  ...    0  2017    0   10.0             0.818565

[5 rows x 23 columns]
      Apr  Aug  Dec  Feb  Jan  Jul  Jun  ...  Sun  Thu  Tue  Wed  year  fog  visib
1348    0    0    0    0    0    0    0  ...    0    0    0    0  2016    0   10.0
2409    0    0    0    0    0    0    0  ...    0    1    0    0  2019    0    9.5
462     1    0    0    0    0    0    0  ...    0    0    0    0  2014    0    7.0
2341    0    0    0    0    0    0    1  ...    0    0    0    0  2019    1    7.9
1738    0    0    0    0    0    0    0  ...    0    0    0    0  2017    0   10.0
2253 

In [191]:
# Create outcome data - targets
targets_fogvis = shuffle_fogvis_train.iloc[:,22]

# check targets data.
print(targets_fogvis[:6])

1348    0.553648
2409    0.609375
462     0.356481
2341    0.680804
1738    0.818565
2253    0.441964
Name: number_of_accidents, dtype: float64


In [192]:
SCALE_num_accidents = 1.0

In [193]:
# Split our data into a training set = 80% of the length of the shuffle array
trainsize = int(len(shuffle_fogvis_train['number_of_accidents'])*0.8)
# The test set size = 20% of the length of the shuffle array.
testsize = len(shuffle_fogvis_train['number_of_accidents']) - trainsize

# Define the number of predictors
nppredictors = 22
# Define the number of targets
noutputs = 1

Create model

In [194]:
# create model
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes previous saved model
shutil.rmtree('/tmp/DNN_regression_trained_model_fv', ignore_errors=True)

estimator_fogvis = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_fv', hidden_units=[20,18,14], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors_fogvis.values)))

# show model is starting to train
print("starting to train");

# Train the model
estimator_fogvis.fit(predictors_fogvis[:trainsize].values, targets_fogvis[:trainsize].values.reshape(trainsize, noutputs)/SCALE_num_accidents, steps=10000)

# check predictions
preds = estimator_fogvis.predict(x=predictors_fogvis[trainsize:].values)

# Apply the Scale value
predslistscale = preds['scores']*SCALE_num_accidents

# Calculate RMSE 
rmse = np.sqrt(np.mean((targets_fogvis[trainsize:].values - predslistscale)**2))
print('DNNRegression fog and visibility has RMSE of {0}'.format(rmse));

# Calculate the mean of the number of accident values.
avg = np.mean(shuffle_fogvis_train['number_of_accidents'][:trainsize])

# Calculate the RMSE using number of accident values and the mean of all target values.
rmse = np.sqrt(np.mean((shuffle_fogvis_train['number_of_accidents'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4f9a88b790>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_fv', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running l

The RMSE values are 0.197 and 0.197 which is excellent (so good in fact that it may indicate an error) so the model will now be tested on the validation data.

Test model on first three rows of validation data

In [195]:
shuffle_fogvis_test[:3]

,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,Fri,Mon,Sat,Sun,Thu,Tue,Wed,year,fog,visib,number_of_accidents
2099,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,2018,0,5.7,0.802691
2429,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,2019,0,10.0,0.555804
2515,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2019,0,10.0,0.765625


In [202]:
# create test using data above
input = pd.DataFrame.from_dict(data = 
				{
         'Apr' : [0,0,0],
         'Aug' : [0,0,0],
         'Dec' : [0,0,1],
         'Feb' : [0,0,0],
         'Jan' : [0,0,0],
         'Jul' : [0,0,0],
         'Jun' : [0,0,0],
         'Mar' : [0,0,0],
         'May' : [0,0,0],
         'Nov' : [0,0,0],
         'Oct' : [1,0,0],
         'Sep' : [0,1,0],
         'Fri' : [0,0,1],
         'Mon' : [0,0,0],
         'Sat' : [1,0,0],
         'Sun' : [0,0,0],
         'Thu' : [0,0,0],
         'Tue' : [0,0,0],
         'Wed' : [0,1,0],
         'year' : [2018,2019,2019],
         'fog' : [0,0,0],
        'visib' : [5.7,10.0,10.0]
        })

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_fv', hidden_units=[20,18,14], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
prednorm = format(str(predslistnorm))
print(prednorm)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4f9a267390>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_fv', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_fv/model.ck

The actual values from the test data are 0.803, 0.556 and 0.766 while the values returned are 0.511, 0.511 and 0.511.  As these values are identical (unlike the previous RMSE values which were just suspiciously close) it appears certain now that there is some error which I am unable to identify, however the model will continue to be tested on a random selection of validation data.

Test the model on three random rows of the validation test set

In [199]:
np.random.seed([11])

shuffle_fogvis_test.sample(n = 3)

,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,Fri,Mon,Sat,Sun,Thu,Tue,Wed,year,fog,visib,number_of_accidents
713,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2014,0,9.8,0.113426
1265,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,2016,1,6.2,0.639485
121,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,2013,0,9.5,0.527523


In [200]:
# create test using data above
input = pd.DataFrame.from_dict(data = 
				{
         'Apr' : [0,0,0],
         'Aug' : [0,0,0],
         'Dec' : [1,0,0],
         'Feb' : [0,0,0],
         'Jan' : [0,0,0],
         'Jul' : [0,1,0],
         'Jun' : [0,0,0],
         'Mar' : [0,0,0],
         'May' : [0,0,1],
         'Nov' : [0,0,0],
         'Oct' : [0,0,0],
         'Sep' : [0,0,0],
         'Fri' : [0,0,0],
         'Mon' : [0,0,0],
         'Sat' : [0,0,0],
         'Sun' : [1,0,0],
         'Thu' : [0,0,1],
         'Tue' : [0,1,0],
         'Wed' : [0,0,0],
         'year' : [2014,2016,2013],
         'fog' : [0,1,0],
         'visib' : [9.8,6.2,9.5]
        })

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_regression_trained_model_fv', hidden_units=[20,18,14], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
prednorm = format(str(predslistnorm))
print(prednorm)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4fa06d9890>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_regression_trained_model_fv', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_regression_trained_model_fv/model.ck

The values from the test data are 0.113, 0.639 and 0.528 while the model still gives the same three identical values showing a definite error.

# Conclusion

**Linear Regression models**

The visibility model showed some promise.  The RMSE values were close as was the result of the testing.  The temperature model showed similarly promising RMSE results but the test results were not accurate.  The fog model also had a good RMSE result but the test results here were more mixed with some very accurate returns and a number less accurate but not too far off.  The fog and temperature model had a fairly close RMSE result though not as close as the previous models.  The test results on the whole were not accurate though one on each set of tests was close.  The model for fog, temperature and visibility did not have a close RMSE result and this was supported by the tests which showed inaccurate results.

**Deep Neural Network models**

The visibility model had a good RMSE result and in the first test showed some very accurate results though this was not repeated in the second test.  The temperature model showed an RMSE result that was not close and consequently, the test results were inaccurate.  The model for fog showed a fairly close RMSE result and similarly, test results that were also quite accurate.  The temperature and visibility model had a good RMSE result and also returned some accurate test results.  However, the model for fog and visibility had an RMSE result that was so good it appeared suspicious.  This was reflected in the testing which returned three identical values in both tests indicating an error in the model.  At the current time this error remains elusively unresolved.

In conclusion it can be seen that the visibility, temperature and fog linear regression models may show some promise in predicting accidents on a particular day.  Furthermore the fog deep neural network model could also prove useful as could the temperature and visibility dnn model.  There may be a chance that the visibility model might also be useful though this would need further investigation.